<h2>Load the data</h2>

In [93]:
import os
import re
import sys
import glob
import numpy as np
import pandas as pd
from nltk import tokenize

In [94]:
txt_files = []
for name in glob.glob('ElecDeb60To16/01*.txt'):
    name = name.replace('\\','/')
    txt_files.append(name)

ann_files = []
for name in glob.glob('ElecDeb60To16/01*.ann'):
    name = name.replace('\\','/')
    ann_files.append(name)

In [98]:
sent_all = []
ind_all = []

for file in txt_files:
    sent_list = []
    ind_list = []
    prev_ind = 0
    
    txt_file = open(file)
    text = txt_file.read()
    txt_file.close()
    
    
    with open(file) as infile:
        for line in infile:
            line = re.sub(r'(\([A-Z]+\)_[A-Z]+(\_[A-Z]+)?: )','',line)
            line = re.sub(r'([A-Z]+\_[A-Z]+\_[A-Z]+: )','',line)
            line = re.sub(r'([A-Z]+\_[A-Z]+: )','',line)
            line = re.sub(r'(\_[A-Z]+: )','',line)
            line = re.sub(r'([A-Z]+: )','',line)
            line = re.sub(r'\[[a-zA-Z0-9;,\s\"\'\.\S]*\]*','',line)
                  
            for sent in tokenize.sent_tokenize(line):
                if not sent_list:
                    ind = text.find(sent,0,len(text)) 
                else:
                    ind = text.find(sent,prev_ind,len(text))
                prev_ind = ind
                sent_list.append([sent,ind])
                ind_list.append(ind)
                
    ind_all.append(np.array(ind_list))
    sent_all.append(sent_list)

#Up to this point we get a sent_all containing every debate (42 total) tokenized into sentences.
#sent_all = [sent_list, sent_list, sent_list,...]
#sent_list = [[sent1, index],[sent2, index],[sent3,index],...]
#ind_all = [ind_list, ind_list, ind_list,...]
#ind_list = [ind, ind, ind,...]

In [96]:
print(ind_all)

[array([    7,    21,   247,   284,   398,   599,   709,   879,   956,
         978,  1098,  1375,  1538,  1717,  1870,  1929,  2037,  2183,
        2308,  2340,  2438,  2472,  2693,  2837,  2986,  3061,  3201,
        3351,  3626,  3843,  3948,  4067,  4128,  4246,  4354,  4671,
        4790,  4868,  5051,  5126,  5230,  5331,  5357,  5414,  5494,
        5510,  5565,  5619,  5672,  5772,  5894,  6143,  6394,  6477,
        6566,  6616,  6648,  6783,  6827,  6894,  6970,  7094,  7156,
        7191,  7427,  7471,  7586,  7647,  7749,  7768,  7835,  7890,
        7963,  7991,  8059,  8236,  8413,  8488,  8560,  8698,  8727,
        8928,  9078,  9114,  9152,  9193,  9314,  9337,  9546,  9583,
        9717,  9878,  9923, 10062, 10211, 10231, 10389, 10421, 10558,
       10581, 10675, 10705, 10758, 11093, 11182, 11255, 11281, 11412,
       11443, 11577, 11608, 11743, 11776, 11838, 11879, 11907, 12016,
       12087, 12152, 12220, 12324, 12420, 12592, 12626, 12836, 12997,
       13091, 13128

In [97]:
ann_all = []

for file in ann_files:
    ann_list = []
    
    with open(file) as infile:
        for line in infile:
            entry = line.split("\t")
            ann_list.append(entry)
    
    ann_all.append(ann_list)

In [57]:
#following part creates labels for task 1 (whether a sentence contains an argument component)

labels_arg_all = []

for i in range(len(ind_all)):
    labels_arg_arr = np.empty([len(ind_all[i]),2])
    sent_start_ind_list = []

    #column 0 of labels_arg are the starting indices of the senteces 
    for j in range(len(ind_all[i])):
        labels_arg_arr[j,0] = ind_all[i][j]
        
    #loop through each annotation entry, get the corresponding sentence starting index 
    for j in range(len(ann_all[i])):
        phrase_start = re.search(r'[0-9]+\s',ann_all[i][j][1]).group(0)
        print(phrase_start)
        sent_start_ind_list.append(ind_all[i][ind_all[i] <= int(phrase_start)].max())
        
        phrase_start = re.search(r';[0-9]+\s',ann_all[i][j][1])
        if phrase_start != None:
            phrase_start = phrase_start.group(0)[1:]
            ind2 = ind_all[i][ind_all[i] <= int(phrase_start)].max()
            if ind2 not in sent_start_ind_list:
                sent_start_ind_list.append(ind2)
    
    #loop through starting indices for sentences, check if each sentence contains an annotation (labeled as 1)
    for j in range(len(ind_all[i])):   
        if ind_all[i][j] in sent_start_ind_list:
            labels_arg_arr[j,1] = 1
        else:
            labels_arg_arr[j,1] = 0
    
    labels_arg_all.append(labels_arg_arr.astype(int))

        
#ann_all = [ann_list, ann_list, ann_list, ...]
#ann_list = [['T1', 'Claim/Premise xxx xxx', '...'], ['T2', 'Claim/Premise xxx xxx', '...'], 
    #['T3', 'Claim/Premise xxx xxx', '...'], ...]
#labels_arg_all = [labels_arg_arr, labels_arg_arr, labesl_arg_arr, ...]
#labels_arg_arr = np array of size (len(ind_list),2), no. of rows = no. of sentences in a txt file, 
    #col 0 = starting index of sentece, col 1 = label (1 for contain claim/premise, 0 for otherwise)

1376 
1154 
1326 
1224 
1934 
1979 
1579 
1718 
1753 
1871 
2038 
2352 
2727 
2767 
2867 
2942 
2987 
3062 
3210 
3255 
3300 
3352 
3458 
3499 
3627 
3776 
3844 
3949 
4068 
4129 
4247 
4605 
4791 
4869 
5052 
5127 
5189 
5231 
5332 
5358 
5495 
5511 
5570 
5624 
6395 
6434 
6482 
6617 
6649 
6696 
6899 
7157 
7192 
7272 
7103 
7303 
7366 
7396 
7428 
7472 
7587 
7750 
7669 
7773 
7836 
8092 
8269 
8309 
8416 
8567 
8730 
8796 
8986 
9159 
9227 
9451 
10162 
9602 
9737 
10065 
10234 
10424 
10584 
10678 
10843 
11005 
11096 
11297 
11459 
11751 
11627 
11942 
12023 
12299 
12246 
12327 
13094 
13131 
13214 
13495 
13548 
15324 
15357 
15412 
15485 
14809 
14919 
14693 
14221 
14265 
16043 
16279 
16340 
16379 
16464 
16523 
17140 
16837 
16882 
16910 
16631 
16762 
17195 
17469 
17583 
17982 
18078 
18289 
18573 
18651 
18970 
19081 
19516 
19675 
19896 
20023 
20461 
20509 
20645 
20686 
20861 
20227 
21174 
21265 
21374 
21523 
21458 
21704 
21791 
21844 
21902 
21963 
22015 
22208 


In [99]:
labels_cp_all = []

for i in range(len(labels_arg_all)):
    labels_cp_list = []
    
    for j in range(labels_arg_all[i].shape[0]):
        if labels_arg_all[i][j,1] != 0:
            labels_cp_list.append(labels_arg_all[i][j,0])
    
    labels_cp_arr = np.zeros((len(labels_cp_list),3))
    labels_cp_out = np.zeros((len(labels_cp_list),2))
    
    for k in range(len(labels_cp_list)):
        labels_cp_arr[k,0] = labels_cp_list[k]

        
    for j in range(len(ann_all[i])):
        #check for claim with only 1 continuous part
        phrase_start = re.search(r'c|Claim\s[0-9]+\s[0-9]+',ann_all[i][j][1])
        if phrase_start != None:
            phrase_start = phrase_start.group(0)
            labels_cp_arr[np.where(labels_cp_arr[:,0] == ind_all[i][ind_all[i] <= int(phrase_start.split()[1])].max())[0],1] +=\
            int(phrase_start.split()[2]) - int(phrase_start.split()[1])
        
        #check for claim formed from 2 separate parts
        phrase_start = re.search(r'c|Claim[0-9\s]+;[0-9]+\s[0-9]+',ann_all[i][j][1])
        if phrase_start != None:
            phrase_start = phrase_start.group(0)
            labels_cp_arr[np.where(labels_cp_arr[:,0] == ind_all[i][ind_all[i] <= int(phrase_start.split(';')[1].split()[0])].max())[0],1] +=\
            int(phrase_start.split(';')[1].split()[1]) - int(phrase_start.split(';')[1].split()[0])
        
        #check for premise with only 1 continuous part
        phrase_start = re.search(r'p|Premise\s[0-9]+\s[0-9]+',ann_all[i][j][1])
        if phrase_start != None:
            phrase_start = phrase_start.group(0)
            labels_cp_arr[np.where(labels_cp_arr[:,0] == ind_all[i][ind_all[i] <= int(phrase_start.split()[1])].max())[0],2] +=\
            int(phrase_start.split()[2]) - int(phrase_start.split()[1])
        
        #check for premise formed from 2 separate parts
        phrase_start = re.search(r'p|Premise[0-9\s]+;[0-9]+\s[0-9]+',ann_all[i][j][1])
        if phrase_start != None:
            phrase_start = phrase_start.group(0)
            labels_cp_arr[np.where(labels_cp_arr[:,0] == ind_all[i][ind_all[i] <= int(phrase_start.split(';')[1].split()[0])].max())[0],2] +=\
            int(phrase_start.split(';')[1].split()[1]) - int(phrase_start.split(';')[1].split()[0])
    
    #compare length of claim and premise components in a sentence, and label the sentence according to the longer component
    labels_cp_out[:,0] = labels_cp_arr[:,0]
    for m in range(labels_cp_arr.shape[0]):
        if labels_cp_arr[m,1] >= labels_cp_arr[m,2]:
            labels_cp_out[m,1] = 1
            
    labels_cp_all.append(labels_cp_out)
    
#labels_cp_all = [labels_cp_out, labels_cp_out, labels_cp_out, ...]
#labels_cp_out = np array of size ((len(labels_cp_list),3)), 
    #no. of rows = no. sentences that contains at least 1 claim or 1 premise,
    #col 0 = starting index of such a sentence, col 1 = label (1 for claim, 0 for premise)

In [59]:
np.set_printoptions(threshold=sys.maxsize)
print(labels_arg_all[0:50])

[array([[    7,     0],
       [   21,     0],
       [  247,     0],
       [  284,     0],
       [  398,     0],
       [  599,     0],
       [  709,     0],
       [  879,     0],
       [  956,     0],
       [  978,     0],
       [ 1098,     1],
       [ 1375,     1],
       [ 1538,     1],
       [ 1717,     1],
       [ 1870,     1],
       [ 1929,     1],
       [ 2037,     1],
       [ 2183,     0],
       [ 2308,     0],
       [ 2340,     1],
       [ 2438,     0],
       [ 2472,     0],
       [ 2693,     1],
       [ 2837,     1],
       [ 2986,     1],
       [ 3061,     1],
       [ 3201,     1],
       [ 3351,     1],
       [ 3626,     1],
       [ 3843,     1],
       [ 3948,     1],
       [ 4067,     1],
       [ 4128,     1],
       [ 4246,     1],
       [ 4354,     1],
       [ 4671,     0],
       [ 4790,     1],
       [ 4868,     1],
       [ 5051,     1],
       [ 5126,     1],
       [ 5230,     1],
       [ 5331,     1],
       [ 5357,     1],
       [ 5